# Template für Topic Modeling
Dieses Template soll dabei helfen, Topic Modeling automatisiert und einheitlich durchzuführen. 

## Allgemeine Vorbereitungsschritte

In [ ]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
# print(os.getcwd())
# os.chdir("C:/SV/HEX/Topic Modeling")

### Pakete laden

In [ ]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
import os
print(os.getcwd())
#os.chdir("C:/Users/mhu/Documents/github/topic_model_it/")
#os.chdir("C:/Users/mhu/Documents/github/HEX_topic_model_it/") 
os.chdir("C:/Users/Hueck/OneDrive/Dokumente/GitHub/HEX_topic_model_it")
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer 
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance

print(f"NumPy Version: {np.__version__}") # Sollte 1.26.x sein
print(f"CUDA verfügbar: {torch.cuda.is_available()}") # Sollte TRUE sein

### Seed setzen
Wir setzen einen festen Seed, um Zufallszahlen in NumPy und PyTorch reproduzierbar zu machen, sowohl auf der CPU als auch auf der GPU (falls verfügbar). Das stellt sicher, dass Berechnungen mit zufälligen Operationen bei wiederholter Ausführung dieselben Ergebnisse liefern.

In [ ]:
seed = 40  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

### Datensätze einlesen
Der Trainings- und der Test-Datensatz werden hier eingelesen. Als Faustregel gilt, der Trainingsdatensatz sollte 80% und der Test-Datensatz 20% des Volumens ausmachen. 
Der Trainings-Datensatz wird für das trainieren / fitten des Modells verwendet. Der Test-Datensatz beinhaltet eine (in diesem Fall manuell erstellte) sogenannte "Ground Truth". Dies ist der Goldstandard, anhand dessen das Modell auf Performance hin überprüft wird. 

In [ ]:
# Training-Datensatz
training_set = pd.read_csv("data/informatikkurse.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame
# training_set = training_set.sample(n=500, random_state=42)  # Zieht eine Zufallsstichprobe von 500 Zeilen aus dem DataFrame mit festgelegtem Seed für Reproduzierbarkeit
training_set = training_set.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
training_set['titel_kursbesch'] = training_set['veranstaltung_titel'] + ' ' + training_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = training_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

In [ ]:
docs

In [ ]:
import pandas as pd

# 1. Daten laden und NaNs bereinigen
df = pd.read_csv("data/labelled_data_final.csv", sep=";").fillna('')

# 2. Kombination und Vorab-Filterung (entfernt leere Synonyme)
df['titel_kursbesch'] = df['veranstaltung_titel'] + ' ' + df['kursbeschreibung']
df = df[df["Synonyme"].str.contains(r'[a-zA-Z0-9]')] # Behält nur Zeilen mit echten Zeichen (isalnum)

# 3. Ground Truth: In einem Rutsch splitten und bereinigen
# Wir nutzen .str.split mit Regex, um Whitespace um Kommas direkt zu entfernen
ground_truth = (
    df["Synonyme"]
    .str.lower()
    .str.split(r'\s*,\s*')
    .apply(lambda terms: [t for t in terms if t.strip()]) # Filtert leere Strings innerhalb der Liste
).tolist()

# 4. Finales Test-Set
test_set = df['titel_kursbesch'].reset_index(drop=True)

In [ ]:
ground_truth

## NLP Vorbereitungsschritte
Zunächst werden die Trainingsdaten eingelesen und die gängigen Vorbereitungsschritte für NLP durchgeführt. Diese wären:
* Stopwords entfernen
* CountVectorizer spezifizieren

### Stopwords entfernen
Im Kontext des hier zu modellierenden Topic Modells werden sowohl standardisierte englische, deutsche als auch individuelle Stopwords generiert und im Objekt `sw` zusammengespielt.
Die Stopwords können je nach Anwendungsfall ergänzt oder reduziert werden.

In [ ]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms) 

In [ ]:
sw

### Lemmatisierung
Durch Lemmatisierung werden die Wörter in einheitliche Begriffe umgewandelt, sodass diese robuster werden. 

## Anwendung: Konfiguration, Training und Evaluation des Topic Models
Hier muss alles innerhalb einer einzigen Code-Zelle erfolgen, da bei allen Konfigurationen variable Parameter vorkommen und wir diese durch das optuna-Package optimieren wollen.

In [ ]:
# Im Objective werden die verschiedenen Parameter-Settings gesetzt, über welche man optimieren möchte
def objective(trial):

  try:
    # Embedding Settings - GPU-beschleunigt
    embedding_model_name = trial.suggest_categorical("embedding_model", ["paraphrase-multilingual-mpnet-base-v2"])
    # UMAP Settings
    n_neighbors = trial.suggest_categorical("n_neighbors", [10, 15, 25])
    min_dist = 0.0
    n_components = trial.suggest_int("n_components", 5, 15, 2)
    # HDBSCAN Settings - KLEINERE Cluster für MEHR spezifische Topics
    min_cluster_size = trial.suggest_int("min_cluster_size", 5, 20, 5)
    min_samples = trial.suggest_int("min_samples", 1, 3, 1)
    # BERTopic Settings - angepasst auf ~40 Labels
    nr_topics = trial.suggest_int("nr_topics", 35, 50, 5)
    diversity = trial.suggest_float("diversity", 0.2, 0.5, step=0.1)
    min_topic_size = trial.suggest_int("min_topic_size", 5, 20, 5)

    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Konfiguration
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # CountVectorizer
    vectorizer = CountVectorizer(
      stop_words=sw,
      token_pattern=r'\b\w+\b',
      ngram_range=(1, 2),
      max_features=10000
    )

    # Embedding Settings - GPU-beschleunigt für RTX 4060
    embedding_model = SentenceTransformer(embedding_model_name, device="cuda")
    
    # UMAP Settings
    umap_model = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric="cosine", random_state=13, low_memory=True)

    # HDBSCAN Settings
    hdbscan_model = HDBSCAN(
      min_cluster_size=min_cluster_size, 
      min_samples=min_samples,
      gen_min_span_tree=False, 
      prediction_data=True
    )

    # Representation Settings
    representation_model = MaximalMarginalRelevance(diversity=diversity)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Training
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic initialisieren
    topic_model = BERTopic(
      embedding_model=embedding_model,
      min_topic_size=min_topic_size,
      nr_topics=nr_topics, 
      language="multilingual",
      umap_model=umap_model,
      vectorizer_model=vectorizer,
      hdbscan_model=hdbscan_model,
      top_n_words=30,
      representation_model=representation_model,
      calculate_probabilities=False,
      low_memory=True
    )

    # BERTopic trainieren
    topic_model_quanten = topic_model.fit(docs)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Topic-Statistik ausgeben
    #--------------------------------------------------------------------------------------------------------------------------------------------
    
    topic_info = topic_model_quanten.get_topic_info()
    num_topics = len(topic_info[topic_info['Topic'] != -1])  # Ohne Outlier
    outlier_count = topic_info[topic_info['Topic'] == -1]['Count'].values[0] if -1 in topic_info['Topic'].values else 0
    outlier_ratio = outlier_count / len(docs)
    
    print(f"\n=== TOPIC STATISTIK ===")
    print(f"Anzahl Topics: {num_topics}")
    print(f"Outlier: {outlier_count} ({outlier_ratio:.1%})")
    print(f"========================\n")
    
    # Trial abbrechen wenn zu wenige Topics (mindestens 15 Topics erforderlich)
    if num_topics < 15:
        print(f"Nur {num_topics} Topics - Trial wird übersprungen")
        raise optuna.TrialPruned()


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Evaluation mit Embedding-basierter Similarity
    #--------------------------------------------------------------------------------------------------------------------------------------------

    from sklearn.metrics.pairwise import cosine_similarity

    # BERTopic auf Test-Daten anwenden
    topics, probs = topic_model_quanten.transform(test_set)
    print(topic_model_quanten.get_topic_freq())

    # Resultierende Topic-Nummern mit den Representations kombinieren
    dataframe_with_results_left = pd.DataFrame(topics, columns=["Topic"])
    dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
    dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

    # Goldstandard abgleichen mit Embedding-Similarity
    similarity_threshold = 0.6  # Schwellenwert für Match (0.6 = moderat, 0.7 = streng)
    metric = 0
    
    for row_number in range(len(ground_truth)):
      ground_truth_current_iteration = ground_truth[row_number]
      result_current_iteration = dataframe_with_results.at[row_number, "Representation"]

      # Embeddings berechnen
      gt_embeddings = embedding_model.encode(ground_truth_current_iteration)
      tm_embeddings = embedding_model.encode(result_current_iteration)
      
      # Cosine Similarity zwischen allen Paaren
      sim_matrix = cosine_similarity(gt_embeddings, tm_embeddings)
      max_sim = sim_matrix.max()
      
      # Match wenn Ähnlichkeit über Schwellenwert
      match_found = max_sim >= similarity_threshold
      if match_found:
        metric += 1

      print("TM:", result_current_iteration[:5], "...")  # Nur erste 5 Wörter
      print("Ground Truth:", ground_truth_current_iteration)
      print(f"Max Similarity: {max_sim:.3f} → {'MATCH ✓' if match_found else 'NO MATCH ✗'}")
      print("---------------------------------------------------------------------------------------------------------------")

    metric_score = metric / len(ground_truth)
    
    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Kombinierte Metrik: Score * Topic-Qualität
    #--------------------------------------------------------------------------------------------------------------------------------------------
    
    # Bestraft Modelle mit weniger als 40 Topics (dein Ziel)
    topic_penalty = min(num_topics / 40, 1.0)
    adjusted_score = metric_score * topic_penalty
    
    print(f"\n=== FINAL ERGEBNIS ===")
    print(f"Raw Score: {metric_score:.3f} ({metric}/{len(ground_truth)} Matches)")
    print(f"Topic Penalty: {topic_penalty:.3f} ({num_topics}/40 Topics)")
    print(f"Adjusted Score: {adjusted_score:.3f}")
    print(f"======================\n")

    return adjusted_score 
  
  except Exception as e:
      print("Trial wird aufgrund eines Errors übersprungen")
      print(f"Verwendete Parameter: embedding model: {embedding_model_name}, n_neighbors: {n_neighbors}, "
            f"min_dist: {min_dist}, n_components: {n_components}, min_cluster_size: {min_cluster_size}, "
            f"min_samples: {min_samples}")
      print(e)
      raise optuna.TrialPruned()

In [23]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=100)

print("Best parameters:", study.best_params)

    Topic  Count
0       0  33991
2      -1   3242
8       1   2678
6       2   2369
4       3   2237
5       4   1969
3       5    960
1       6    703
7       7    616
11      8    412
12      9    311
9      10    291
13     11    265
14     12    264
17     13    235
18     14    221
10     15     52
34     16     44
16     17     44
27     18     39
23     19     38
15     20     37
26     21     36
24     22     35
32     23     25
20     24     23
19     25     20
21     26     15
28     27     15
33     28     15
29     29     12
30     30     11
31     31     11
22     32      7
25     33      5
TM: ['data', 'course', 'systems', 'security', 'software'] ...
Ground Truth: ['software development', 'software design', 'software architecture', 'application development', 'software engineering', 'softwareentwicklung', 'softwaredesign', 'softwarearchitektur', 'anwendungsentwicklung']
Max Similarity: 0.906 → MATCH ✓
-----------------------------------------------------------------------

[I 2026-02-25 20:20:05,448] Trial 0 finished with value: 0.8576195773081201 and parameters: {'embedding_model': 'paraphrase-multilingual-mpnet-base-v2', 'n_neighbors': 15, 'n_components': 11, 'min_cluster_size': 5, 'min_samples': 1, 'nr_topics': 35, 'diversity': 0.5, 'min_topic_size': 30}. Best is trial 0 with value: 0.8576195773081201.


TM: ['informatik', 'learning', 'systems', 'engineering', 'methoden'] ...
Ground Truth: ['requirements analysis', 'software requirements', 'requirements specification', 'requirements management', 'anforderungsengineering', 'anforderungsanalyse', 'softwareanforderungen', 'anforderungsspezifikation', 'anforderungsmanagement']
Max Similarity: 0.666 → MATCH ✓
---------------------------------------------------------------------------------------------------------------
TM: ['informatik', 'learning', 'systems', 'engineering', 'methoden'] ...
Ground Truth: ['it law', 'technology ethics', 'digital law', 'it ethics', 'it-recht und ethik', 'technologierecht', 'digitale ethik', 'it-ethik', 'informationsrecht']
Max Similarity: 0.655 → MATCH ✓
---------------------------------------------------------------------------------------------------------------
TM: ['pattern recognition', 'signal processing', 'filter', 'bayes', 'classification'] ...
Ground Truth: ['signal processing', 'digital signal proce

[I 2026-02-25 20:23:03,988] Trial 1 finished with value: 0.8954393770856507 and parameters: {'embedding_model': 'paraphrase-multilingual-mpnet-base-v2', 'n_neighbors': 25, 'n_components': 13, 'min_cluster_size': 10, 'min_samples': 1, 'nr_topics': 35, 'diversity': 0.30000000000000004, 'min_topic_size': 25}. Best is trial 1 with value: 0.8954393770856507.


TM: ['informatik', 'software', 'algorithmen', 'programmierung', 'systeme'] ...
Ground Truth: ['big data analytics', 'data analytics', 'data mining', 'data engineering', 'data science und big data', 'big-data-analyse', 'datenanalyse', 'data mining', 'datenengineering']
Max Similarity: 0.781 → MATCH ✓
---------------------------------------------------------------------------------------------------------------
TM: ['signal', 'signal processing', 'transmission', 'communication systems', 'filter'] ...
Ground Truth: ['theory of computation', 'computational theory', 'formal methods', 'automata theory', 'theoretische informatik', 'berechenbarkeitstheorie', 'computertheorie', 'formale methoden', 'automatentheorie']
Max Similarity: 0.537 → NO MATCH ✗
---------------------------------------------------------------------------------------------------------------
TM: ['informatik', 'software', 'algorithmen', 'programmierung', 'systeme'] ...
Ground Truth: ['algorithm design', 'data organization', 

[W 2026-02-25 20:24:44,897] Trial 2 failed with parameters: {'embedding_model': 'paraphrase-multilingual-mpnet-base-v2', 'n_neighbors': 25, 'n_components': 5, 'min_cluster_size': 10, 'min_samples': 2, 'nr_topics': 40, 'diversity': 0.5, 'min_topic_size': 15} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\Programme\miniconda\envs\hex_topic_model_it\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Hueck\AppData\Local\Temp\ipykernel_26192\1621151099.py", line 69, in objective
    topic_model_quanten = topic_model.fit(docs)
                          ^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programme\miniconda\envs\hex_topic_model_it\Lib\site-packages\bertopic\_bertopic.py", line 392, in fit
    self.fit_transform(documents=documents, embeddings=embeddings, y=y, images=images)
  File "d:\Programme\miniconda\envs\hex_topic_model_it\Lib\site-pack

KeyboardInterrupt: 

In [ ]:
# Top 10 Trials anzeigen
import pandas as pd

# Alle Trials als DataFrame
trials_df = study.trials_dataframe()

# Nach Score sortieren (absteigend) und Top 10 anzeigen
top_10 = trials_df.nlargest(10, 'value')[['number', 'value', 'params_n_neighbors', 'params_n_components', 
                                           'params_min_cluster_size', 'params_min_samples', 
                                           'params_nr_topics', 'params_diversity', 'params_min_topic_size']]
print("=== TOP 10 TRIALS ===")
print(top_10.to_string())

# Oder kompakter:
print("\n=== TOP 10 BESTE PARAMETER ===")
for i, trial in enumerate(sorted(study.trials, key=lambda t: t.value if t.value else 0, reverse=True)[:10]):
    print(f"{i+1}. Trial {trial.number}: Score={trial.value:.3f}")
    print(f"   {trial.params}")
    print()